In [1]:
import pandas as pd
pd.set_option('display.float_format', lambda x: '%.2f' % x)
import numpy as np
from google.cloud import bigquery
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set_style('darkgrid')
import joblib
import warnings
warnings.filterwarnings('ignore') 
import re

bq_client = bigquery.Client()

In [2]:
import google.auth
from google.cloud import bigquery
from google.cloud import bigquery_storage
import time

bqclient = bigquery.Client()
bqstorageclient = bigquery_storage.BigQueryReadClient()

## Exploring subscriber data for 'unlimited' subscription type 

* **Make data set for EDA**
    * Fetching GA data
    
        * SUBSCRIBERS:
            * Subscriber id > fullvid > session > pagePaths > GA data for pagePaths <BR><BR>
        * NON-SUBSCRIBERS:
            * fullvid > session > pagePaths > GA data for pagePaths

NOTE: 
* Data not grouped by anything. For respective features, group by during dev time
* Data not filtered by any date range - decide apporpriate date filtering during eda

In [3]:
start_time = time.time()

query_string = """

CREATE OR REPLACE TABLE `api-project-901373404215.skt.content_articles` AS (

   # Get IAB Tiers for articles. Articles can be classified multiple times with different tier1 values given. 
   # To avoid duplication, partition by naturalid and take categorization by latest timestamp
   
   WITH content_iab AS (
   SELECT * FROM (
        SELECT 
            naturalId AS iab_natid, 
            RANK() OVER (PARTITION BY naturalid ORDER BY timestamp DESC) AS mostrecent,
            categoryName,
            tier1,
            tier2
        FROM `api-project-901373404215.Content.mnet_iab_categories` where tier1 IS NOT NULL
        ) 
    WHERE mostrecent = 1
    ),

    # Get the natid and the corresponding title form the main content table. There are duplicates so partition by naturalid and get latest articles 
    
    content AS (
      SELECT 
        natid AS content_natid,
        title,
        publish_date,
        body
      FROM (
        SELECT DISTINCT
          LOWER(NaturalId) AS natid,
          RANK() OVER (PARTITION BY naturalid ORDER BY timestamp DESC) AS mostrecent,
          title,
          DATETIME(date, "America/New_York") as publish_date,
          body
        FROM
          `api-project-901373404215.Content.content`
        WHERE
          Visible is true
          AND type in (
          'blog',
          'blogslide',
          'magazine')
          )
      WHERE mostrecent = 1
    )
    
    # Join the content iab table with the main content table to get the title with the iab category
      SELECT
        * 
        EXCEPT (iab_natid, mostrecent)
      FROM
        content
      LEFT JOIN
        content_iab
      ON
        LOWER(content_natid) = LOWER(iab_natid)
)
"""

content = (
    bqclient.query(query_string)
    .result()
    .to_dataframe(bqstorage_client=bqstorageclient)
) 

print("--- %s seconds ---" % (time.time() - start_time))

--- 54.823782444000244 seconds ---


In [4]:
start_time = time.time()

query_string = """
  SELECT
       content_natid, title
    FROM
        `api-project-901373404215.skt.content_articles`
"""
curios = (
    bqclient.query(query_string)
    .result()
    .to_dataframe(bqstorage_client=bqstorageclient)
) 

print("--- %s seconds ---" % (time.time() - start_time))

print(curios.shape)

--- 9.162334680557251 seconds ---
(1024511, 2)


#### --SUBSCRIBERS--
* **Make subscribers pool - raw IDs dataset**

In [5]:
start_time = time.time()

query_string = """

CREATE OR REPLACE TABLE `api-project-901373404215.skt.raw_subscriber` AS

    # get ALL user_id_uids having unlimited type subscription only
    WITH eligible_users AS ( 
      
       SELECT 
           DISTINCT 
               user_id_uid, 
               resource_id_rid, 
               start_date,
               status, 
               subscription_trial_end_date
               
               # filter for active only
               --AND status='active'
               --AND total__refunded<1
               --AND cast(dt_updated as date)=current_date('America/New_York')
    FROM
        `api-project-901373404215.piano.subscriber_details`
    WHERE 
        # Filter for the 'universal' subscriptions only
        resource_id_rid IN UNNEST(['RKPEVDB', 'R8W03AS'])
        )
    
    # join user_id_uids with GA's pianoIDs and For each pianoID, get their fullvids. De-duplicate.
     SELECT 
         DISTINCT
            ga_pianoId,
            user_id_uid,
            ga_fullvisitorid,
            resource_id_rid,
            start_date,
            status, 
            subscription_trial_end_date
    FROM
        eligible_users
         INNER JOIN 
         `api-project-901373404215.DataMart.v_DataMart_updated` 
    ON 
        LOWER(ga_pianoId) = LOWER(user_id_uid)
"""


make_data = (
    bqclient.query(query_string)
    .result()
    .to_dataframe(bqstorage_client=bqstorageclient)
) 

print("--- %s seconds ---" % (time.time() - start_time))

--- 13.796920537948608 seconds ---


* Query raw subscribers data (sanity check)

In [3]:
start_time = time.time()

query_string = """
  SELECT
       *
    FROM
        `api-project-901373404215.skt.raw_subscriber`
"""
raw = (
    bqclient.query(query_string)
    .result()
    .to_dataframe(bqstorage_client=bqstorageclient)
) 

print("--- %s seconds ---" % (time.time() - start_time))

print(raw.shape)

--- 2.9498701095581055 seconds ---
(130322, 7)


In [7]:
print("Unique unlimited subscribers: ", len(raw.user_id_uid.unique()))
print("Unique unlimited fullvids: ", len(raw.ga_fullvisitorid.unique()), "\n")
print("Duplicates?: ", raw.duplicated().any(), "\n")
raw.head()

Unique unlimited subscribers:  42678
Unique unlimited fullvids:  101600 

Duplicates?:  False 



,ga_pianoId,user_id_uid,ga_fullvisitorid,resource_id_rid,start_date,status,subscription_trial_end_date
0,pniajfki7qfd903,PNIajFKi7qfd903,5815314647275138317,R8W03AS,2020-11-01 09:17:07 -0500,active,None
1,pniajfki7qfd903,PNIajFKi7qfd903,5815314647275138317,R8W03AS,2020-11-01 09:17:07 -0500,expired,None
2,pniajfki7qfd903,PNIajFKi7qfd903,3242418031940008152,R8W03AS,2020-11-01 09:17:07 -0500,expired,None
3,pniajfki7qfd903,PNIajFKi7qfd903,3242418031940008152,R8W03AS,2020-11-01 09:17:07 -0500,active,None
4,pniajfki7qfd903,PNIajFKi7qfd903,16644116536832264178,R8W03AS,2020-11-01 09:17:07 -0500,expired,None


In [8]:
# every piano ID has on average 2 fullvids

pd.DataFrame(raw.groupby('ga_pianoId').ga_fullvisitorid.nunique().describe())

,ga_fullvisitorid
count,42678.00
mean,2.39
std,4.96
min,1.00
25%,1.00
50%,1.00
75%,2.00
max,205.00


In [9]:
print("Min subscription start", raw.start_date.min())
print("Max subscription start", raw.start_date.max())

Min subscription start 2020-11-01 09:17:07 -0500
Max subscription start 2021-11-09 02:10:13 -0500


In [4]:
# EXTRA

# query takes all the 42k users for EDA (despite account status) because they subscribed in the first place so behavior may matter

raw_acc = raw[['user_id_uid', 'status']].drop_duplicates(keep='first')
print(":: Types of account status ::")
print(raw_acc.status.value_counts(), "\n")

print(":: Nulls in whole data ::")
print(raw.isna().sum(), "\n")

# duplicate pianoid -fullvid rows present :: because for those combo - multiple account info presnt

inspect = pd.DataFrame(raw.groupby(['user_id_uid', 'ga_fullvisitorid']).size()).reset_index().rename(columns={0:'size'})

# inspect[inspect['size']>1]

# print(":: Duplicate pianoID - fvid entries present ::")
# raw[(raw.user_id_uid=='29QXHHnco3XmrcrwNTxoP8p687x1') & (raw.ga_fullvisitorid =='1225771203912128738')]

:: Types of account status ::
active             42310
cancelled           5213
payment failure     3054
expired             2282
upgraded              14
Name: status, dtype: int64 

:: Nulls in whole data ::
ga_pianoId                          0
user_id_uid                         0
ga_fullvisitorid                    0
resource_id_rid                     0
start_date                          0
status                              0
subscription_trial_end_date    120363
dtype: int64 



* **Pull GA data for these subscribers (no date limit)**

In [11]:
start_time = time.time()

query_string = """ 

CREATE OR REPLACE TABLE `api-project-901373404215.skt.subscriber_ga_data` AS (
  
    # get article content + IAB tiers
    WITH content_joined AS (
      SELECT
        *
      FROM
        `api-project-901373404215.skt.content_articles`
    )
    
    # for all eligible subscribers - fetch their GA data for preparing features
    SELECT 
            z.ga_pianoId,
            user_id_uid,
            resource_id_rid,
            d.GA_fullVisitorId, 
            GA_visitStartTime, 
            GA_date, 
            GA_pagePath, 
            -- feature cols below
            GA_dfpNewZone, GA_visitNumber,
            GA_pageViews, GA_scrollDepth, timeOnPage,  
            GA_cmsNaturalId, title, publish_date, 
            GA_deviceOperatingSystem, GA_deviceCategory, GA_deviceBrowser,
            GA_country, GA_referralGroup,
            GA_primaryChannel, GA_primarySection, tier1, tier2
 
    FROM 
        `api-project-901373404215.skt.raw_subscriber` z
    INNER JOIN 
        `api-project-901373404215.DataMart.v_DataMart_updated` d
    ON 
        z.ga_fullvisitorid = d.GA_fullVisitorId
    LEFT JOIN
            content_joined cj
        ON d.GA_cmsNaturalId = cj.content_natid
)
     """

make_subs_data = (
    bqclient.query(query_string)
    .result()
    .to_dataframe(bqstorage_client=bqstorageclient)
) 

print("--- %s seconds ---" % (time.time() - start_time))

--- 51.89879870414734 seconds ---


* Query subscribers' GA data (sanity check)

In [4]:
start_time = time.time()

query_string = """
  SELECT
       *
    FROM
        `api-project-901373404215.skt.subscriber_ga_data`
"""

subs_data = (
    bqclient.query(query_string)
    .result()
    .to_dataframe(bqstorage_client=bqstorageclient)
) 

print("--- %s seconds ---" % (time.time() - start_time))



--- 21.606513023376465 seconds ---


In [5]:
print(subs_data.shape)

(9746613, 24)


In [6]:
print("Unique unlimited subscribers: ", len(subs_data.user_id_uid.unique()))
print("Unique unlimited fullvids: ", len(subs_data.GA_fullVisitorId.unique()), "\n")
# print("Duplicates?: ", subs_data.duplicated().any())

subs_data.head()

Unique unlimited subscribers:  42678
Unique unlimited fullvids:  101600 



,ga_pianoId,user_id_uid,resource_id_rid,GA_fullVisitorId,GA_visitStartTime,GA_date,GA_pagePath,GA_dfpNewZone,GA_visitNumber,GA_pageViews,...,publish_date,GA_deviceOperatingSystem,GA_deviceCategory,GA_deviceBrowser,GA_country,GA_referralGroup,GA_primaryChannel,GA_primarySection,tier1,tier2
0,pniky78aiqkzet5,PNIky78Aiqkzet5,R8W03AS,2385517737961160662,1623864308,2021-06-16,/sites/lucianapaulise/2021/06/16/these-are-the...,article/standard/subscriber/alx,94,1,...,2021-06-16 09:00:00,macintosh,desktop,chrome,united states,organic search,leadership,careers,Business and Finance,Industries
1,pni7ktmmtqv626w,PNI7ktMmTqv626w,R8W03AS,215024215001268946,1634312293,2021-10-15,/sites/kwamechristian/2021/10/14/got-diversity...,article/standard/subscriber/alx,37,1,...,2021-10-14 22:57:04,macintosh,desktop,chrome,united states,organic search,leadership,careers,Business and Finance,Business
2,pniv9mzygqjw92n,PNIV9mzygqjw92n,R8W03AS,7551813098065067552,1626107338,2021-07-12,/sites/margiewarrell/2021/01/06/does-a-she-ces...,article/standard/subscriber/alx,21,1,...,2021-01-06 11:23:12,macintosh,desktop,chrome,united states,organic search,leadership,careers,Careers,Remote Working
3,pniwa5hb5qmfq6z,PNIWa5hb5qmfq6z,R8W03AS,2033550442267254422,1624619833,2021-06-25,/sites/tracybrower/2021/04/18/women-and-the-pa...,article/standard/subscriber/alx,261,1,...,2021-04-18 17:50:00,windows,desktop,chrome,united states,organic search,leadership,careers,Business and Finance,Business
4,pnifp6eosqjncbn,PNIfP6EOsqjncbn,R8W03AS,9177976390165660544,1615306371,2021-03-09,/sites/danabrownlee/2020/07/14/got-bored-kids-...,article/standard/subscriber/alx,1057,1,...,2020-07-14 06:50:00,windows,desktop,firefox,united states,organic search,leadership,careers,Hobbies & Interests,Games and Puzzles


In [15]:
subs_data.isna().sum()

ga_pianoId                        0
user_id_uid                       0
resource_id_rid                   0
GA_fullVisitorId                  0
GA_visitStartTime                 0
GA_date                           0
GA_pagePath                       0
GA_dfpNewZone                522108
GA_visitNumber                    0
GA_pageViews                      0
GA_scrollDepth              1596742
timeOnPage                    73063
GA_cmsNaturalId                  62
title                       3477623
publish_date                3477622
GA_deviceOperatingSystem          0
GA_deviceCategory                 0
GA_deviceBrowser                  0
GA_country                        0
GA_referralGroup                  0
GA_primaryChannel                 0
GA_primarySection                 0
tier1                       4967347
tier2                       5357351
dtype: int64

In [7]:
print(subs_data.GA_date.min())
print(subs_data.GA_date.max())

2018-01-01
2021-11-08


* Why no date filter in above query? - May offer more flexibility in experimenting with users' histories falling in different date ranges, such as:

    * full history of user
        * consists of all before + all after subscription user's behavior - obscured <br><br>
    * ga_date > jan 1, '21 
        * may consist of some before + all after subscription behavior - obscured<br><br>
    * ga_date within (subscribed date - 90 days)
        * consists of before/after subscription mutually exclusive behavior per person - hypothesis: something they did in pasy 90 days which led them to subsribe)
        * will need Python function like below or data pipeline per Greg

In [17]:
# just a prototype example on 1 user -- getting data for ga_date within (subscribed date - 90 days)

raw[raw.ga_pianoId =='pniimmquxqxev7l']

,ga_pianoId,user_id_uid,ga_fullvisitorid,resource_id_rid,start_date,status,subscription_trial_end_date
124704,pniimmquxqxev7l,PNIIMmQuXqxev7l,3663613126660091315,RKPEVDB,2021-08-06 05:20:00 -0400,active,None
124705,pniimmquxqxev7l,PNIIMmQuXqxev7l,6489370869177338808,RKPEVDB,2021-08-06 05:20:00 -0400,active,None
124706,pniimmquxqxev7l,PNIIMmQuXqxev7l,330224175711610250,RKPEVDB,2021-08-06 05:20:00 -0400,active,None
124707,pniimmquxqxev7l,PNIIMmQuXqxev7l,3237950183657409267,RKPEVDB,2021-08-06 05:20:00 -0400,active,None
124708,pniimmquxqxev7l,PNIIMmQuXqxev7l,3499397066445014412,RKPEVDB,2021-08-06 05:20:00 -0400,active,None
124709,pniimmquxqxev7l,PNIIMmQuXqxev7l,8210077433885756007,RKPEVDB,2021-08-06 05:20:00 -0400,active,None
124710,pniimmquxqxev7l,PNIIMmQuXqxev7l,2446258376655790451,RKPEVDB,2021-08-06 05:20:00 -0400,active,None
124711,pniimmquxqxev7l,PNIIMmQuXqxev7l,2125747004827475854,RKPEVDB,2021-08-06 05:20:00 -0400,active,None
124712,pniimmquxqxev7l,PNIIMmQuXqxev7l,1961813583447824657,RKPEVDB,2021-08-06 05:20:00 -0400,active,None
124713,pniimmquxqxev7l,PNIIMmQuXqxev7l,6255747600586462916,RKPEVDB,2021-08-06 05:20:00 -0400,active,None


In [18]:
# showing example on a radom user
user = subs_data[subs_data.ga_pianoId == 'pniimmquxqxev7l']

# user came on all following dates
user.GA_date.drop_duplicates(keep='first').reset_index(drop=True).sort_values()

439    2019-05-31
504    2019-06-01
760    2019-06-02
814    2019-06-03
92     2019-06-04
          ...    
722    2021-11-04
4      2021-11-05
2      2021-11-06
1      2021-11-07
407    2021-11-08
Name: GA_date, Length: 834, dtype: object

In [19]:
import datetime

user.GA_date = pd.to_datetime(user.GA_date)

# get user's subscription date
user['start_date'] = '2021-08-06'
user.start_date = pd.to_datetime(user.start_date)

# calculate cutoff date i.e. subscription date - 90 days
user['cutoff_90day_dt'] = user.start_date - datetime.timedelta(days=90)

# select user's GA data falling within 90 days before subscription -- MAKE FEATURES OUT OF THESE
user[(user.GA_date >=user.cutoff_90day_dt)  & (user.GA_date <user.start_date)].sort_values('GA_date')

,ga_pianoId,user_id_uid,resource_id_rid,GA_fullVisitorId,GA_visitStartTime,GA_date,GA_pagePath,GA_dfpNewZone,GA_visitNumber,GA_pageViews,...,GA_deviceCategory,GA_deviceBrowser,GA_country,GA_referralGroup,GA_primaryChannel,GA_primarySection,tier1,tier2,start_date,cutoff_90day_dt
8185117,pniimmquxqxev7l,PNIIMmQuXqxev7l,RKPEVDB,1961813583447824657,1620485482,2021-05-08,/,none,2984,1,...,desktop,chrome,united kingdom,organic search,home,none,None,None,2021-08-06,2021-05-08
6851888,pniimmquxqxev7l,PNIIMmQuXqxev7l,RKPEVDB,1961813583447824657,1620508620,2021-05-08,/home_usa/,none,2985,1,...,desktop,chrome,united kingdom,organic search,home,none,None,None,2021-08-06,2021-05-08
8689436,pniimmquxqxev7l,PNIIMmQuXqxev7l,RKPEVDB,1961813583447824657,1620485482,2021-05-08,/home_usa/,none,2984,1,...,desktop,chrome,united kingdom,organic search,home,none,None,None,2021-08-06,2021-05-08
6944775,pniimmquxqxev7l,PNIIMmQuXqxev7l,RKPEVDB,1961813583447824657,1620485482,2021-05-08,/sites/alexknapp/2021/05/08/barney-ford-man-wh...,article/premium/default/standard,2984,1,...,desktop,chrome,united kingdom,organic search,innovation,science,Business and Finance,None,2021-08-06,2021-05-08
7163606,pniimmquxqxev7l,PNIIMmQuXqxev7l,RKPEVDB,1961813583447824657,1620508620,2021-05-08,/worlds-billionaires/,none,2985,1,...,desktop,chrome,united kingdom,organic search,billionaires,none,None,None,2021-08-06,2021-05-08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9742518,pniimmquxqxev7l,PNIIMmQuXqxev7l,RKPEVDB,1961813583447824657,1628072296,2021-08-04,/sites/angelauyeung/2021/08/03/as-biology-manu...,article/masthead/default/standard,3293,1,...,desktop,chrome,united kingdom,organic search,billionaires,none,Business and Finance,Industries,2021-08-06,2021-05-08
1828868,pniimmquxqxev7l,PNIIMmQuXqxev7l,RKPEVDB,1961813583447824657,1628191917,2021-08-05,/home_usa/,none,3298,1,...,desktop,chrome,united kingdom,organic search,home,none,None,None,2021-08-06,2021-05-08
3498156,pniimmquxqxev7l,PNIIMmQuXqxev7l,RKPEVDB,1961813583447824657,1628165866,2021-08-05,/worlds-billionaires/,none,3297,1,...,desktop,chrome,united kingdom,organic search,billionaires,none,None,None,2021-08-06,2021-05-08
4017109,pniimmquxqxev7l,PNIIMmQuXqxev7l,RKPEVDB,1961813583447824657,1628165866,2021-08-05,/,none,3297,1,...,desktop,chrome,united kingdom,organic search,home,none,None,None,2021-08-06,2021-05-08


#### --NON-SUBSCRIBERS--
* **Make non-subscriber pool - raw IDs datasets**
    * eligible fvid = those w/ count (articles) > 4 in a month this year

In [3]:
start_time = time.time()

query_string = """
    CREATE OR REPLACE TABLE `api-project-901373404215.skt.raw_oct_ns` AS
    (
    SELECT
                rand() AS random_num,
                piano_id,
                ga_fullvisitorid
    FROM (
                SELECT
                    ga_fullvisitorid, 
                    EXTRACT(MONTH FROM ga_date) AS ga_month,
                    MAX(ga_pianoId) AS piano_id,
                    count(GA_cmsNaturalId) AS num_articles
                FROM
                    `api-project-901373404215.DataMart.v_DataMart_updated`
                WHERE 
                    ga_date BETWEEN '2021-10-01' AND '2021-10-31' AND ga_fullvisitorid NOT IN 
                        (
                        SELECT DISTINCT ga_fullvisitorid FROM `api-project-901373404215.skt.raw_subscriber`
                        )
                    AND STARTS_WITH(GA_cmsNaturalId, "blogandpostid/blog/post/")
                
                GROUP BY 
                    ga_fullvisitorid, ga_month
                HAVING 
        
                    # make sure the fvid never had a piano id and has read >4 articles in a month this year
                    
                    piano_id IS NULL AND 
                    num_articles > 4
    )
    ORDER BY random_num
    LIMIT 110000
    )
"""
                    

make_ns_data = (
    bqclient.query(query_string)
    .result()
    .to_dataframe(bqstorage_client=bqstorageclient)
) 

print("--- %s seconds ---" % (time.time() - start_time)) # took 50sec

--- 11.6127028465271 seconds ---


* Query raw non-subscribers IDs' data (sanity check)

In [4]:
start_time = time.time()

query_string = """
  SELECT
       *
    FROM
        `api-project-901373404215.skt.raw_oct_ns`
"""
ns = (
    bqclient.query(query_string)
    .result()
    .to_dataframe(bqstorage_client=bqstorageclient)
) 

print("--- %s seconds ---" % (time.time() - start_time))

print(ns.shape)

--- 3.679318904876709 seconds ---
(110000, 3)


In [5]:
ns.isna().sum()

random_num               0
piano_id            110000
ga_fullvisitorid         0
dtype: int64

In [6]:
ns_fvids = list(ns.ga_fullvisitorid)
s_fvids = list(raw.ga_fullvisitorid.unique())

print("subscribed fvids: ", len(s_fvids))
print("non-subscribed fvids: ", len(ns_fvids))

print("any intersection between them?: ", list(set(s_fvids) & set(ns_fvids)))

NameError: name 'raw' is not defined

* **Pull GA data for these non-subscribers (no date limit)**

In [6]:
start_time = time.time()

query_string = """ 
CREATE OR REPLACE TABLE `api-project-901373404215.skt.oct_ns_ga_data` AS (

    # get article content + IAB tiers
    WITH content_joined AS (
      SELECT
        *
      FROM
        `api-project-901373404215.skt.content_articles`
    )

    # for sampled non-subscribers - fetch their GA data for preparing features
    SELECT 
            z.piano_id,
            d.GA_fullVisitorId, 
            GA_visitStartTime,
            GA_date, 
            GA_pagePath,             
            -- feature cols below
            GA_dfpNewZone, GA_visitNumber,
            GA_pageViews, GA_scrollDepth, timeOnPage,  
            GA_cmsNaturalId, title, publish_date, 
            GA_deviceOperatingSystem, GA_deviceCategory, GA_deviceBrowser,
            GA_country, GA_referralGroup,
            GA_primaryChannel, GA_primarySection, tier1, tier2
  
    FROM 
        `api-project-901373404215.skt.raw_oct_ns` z
    INNER JOIN 
        `api-project-901373404215.DataMart.v_DataMart_updated` d
    ON 
        z.ga_fullvisitorid = d.GA_fullVisitorId
    LEFT JOIN
            content_joined cj
        ON d.GA_cmsNaturalId = cj.content_natid
)
     """

make_nsga_data = (
    bqclient.query(query_string)
    .result()
    .to_dataframe(bqstorage_client=bqstorageclient)
) 

print("--- %s seconds ---" % (time.time() - start_time))

--- 47.735273599624634 seconds ---


* Query non-subscribers' GA data (sanity check)

In [7]:
start_time = time.time()

query_string = """
  SELECT
       *
    FROM
        `api-project-901373404215.skt.oct_ns_ga_data`
"""

nonsubs_data = (
    bqclient.query(query_string)
    .result()
    .to_dataframe(bqstorage_client=bqstorageclient)
) 

print("--- %s seconds ---" % (time.time() - start_time))

--- 29.256755590438843 seconds ---


In [8]:
print(nonsubs_data.shape)

(4293776, 22)


In [9]:
print("Unique unlimited fullvids: ", len(nonsubs_data.GA_fullVisitorId.unique()), "\n")
# print("Duplicates?: ", nonsubs_data.duplicated().any())

nonsubs_data.head()

Unique unlimited fullvids:  110000 



,piano_id,GA_fullVisitorId,GA_visitStartTime,GA_date,GA_pagePath,GA_dfpNewZone,GA_visitNumber,GA_pageViews,GA_scrollDepth,timeOnPage,...,publish_date,GA_deviceOperatingSystem,GA_deviceCategory,GA_deviceBrowser,GA_country,GA_referralGroup,GA_primaryChannel,GA_primarySection,tier1,tier2
0,None,10557746298323887606,1589131393,2020-05-10,/sites/billybambrough/2020/05/09/detached-from...,article-amp/standard/default/standard,81,1,0.50,42.00,...,2020-05-09 19:37:07,android,mobile,android webview,united states,content aggregators,money,crypto & blockchain,Personal Finance,Financial Assistance
1,None,5517186610130244420,1595743126,2020-07-26,/sites/ashleaebeling/2020/07/24/republican-sti...,article-amp/standard/default/standard,126,1,0.50,83.00,...,2020-07-24 15:18:48,android,mobile,android webview,united states,content aggregators,money,personal finance,Personal Finance,Financial Assistance
2,None,14031961478331278393,1564764330,2019-08-02,/sites/suzannerowankelleher/2019/08/01/everyon...,None,86,1,0.00,nan,...,2019-08-01 13:42:23,android,mobile,android webview,united states,content aggregators,lifestyle,travel,Personal Finance,Financial Assistance
3,None,6555035674888858043,1586716998,2020-04-12,/sites/zackfriedman/2020/04/12/stimulus-check-...,article-amp/standard/default/standard,117,1,0.25,9.00,...,2020-04-12 12:30:02,android,mobile,android webview,united states,content aggregators,money,personal finance,Personal Finance,Financial Assistance
4,None,16609829608504750676,1597407383,2020-08-14,/sites/shaharziv/2020/08/13/confirmed-extra-40...,article-amp/standard/default/standard,83,1,0.25,17.00,...,2020-08-13 09:03:26,android,mobile,android webview,united states,content aggregators,money,personal finance,Personal Finance,Financial Assistance


In [10]:
nonsubs_data.isna().sum()

piano_id                    4293776
GA_fullVisitorId                  0
GA_visitStartTime                 0
GA_date                           0
GA_pagePath                       0
GA_dfpNewZone                633073
GA_visitNumber                    0
GA_pageViews                      0
GA_scrollDepth               118425
timeOnPage                   357138
GA_cmsNaturalId                 398
title                         34026
publish_date                  34026
GA_deviceOperatingSystem          0
GA_deviceCategory                 0
GA_deviceBrowser                  0
GA_country                        0
GA_referralGroup                  0
GA_primaryChannel                 0
GA_primarySection                 0
tier1                       1302593
tier2                       1484523
dtype: int64

In [11]:
print(nonsubs_data.GA_date.min())
print(nonsubs_data.GA_date.max())

2018-01-01
2021-11-08


* Double checking no subs included in non-subs

In [12]:
nonsubs_data.GA_dfpNewZone = nonsubs_data.GA_dfpNewZone.fillna('none')

# maybe Forbes people?
nonsubs_data[nonsubs_data.GA_dfpNewZone.str.contains('/subscriber/')]

,piano_id,GA_fullVisitorId,GA_visitStartTime,GA_date,GA_pagePath,GA_dfpNewZone,GA_visitNumber,GA_pageViews,GA_scrollDepth,timeOnPage,...,publish_date,GA_deviceOperatingSystem,GA_deviceCategory,GA_deviceBrowser,GA_country,GA_referralGroup,GA_primaryChannel,GA_primarySection,tier1,tier2
384,None,16340644851760120304,1605814398,2020-11-19,/sites/elanagross/2020/11/19/ben-carson-says-h...,article-amp/topline/subscriber/standard,68,1,0.00,0.00,...,2020-11-19 12:40:28,android,mobile,android webview,united states,content aggregators,business,none,Medical Health,Diseases and Conditions
7819,None,17909298741426735291,1611725731,2021-01-27,/sites/paultassi/2021/01/26/destiny-2-reset-up...,article-amp/standard/subscriber/standard,70,1,0.50,119.00,...,2021-01-26 12:37:19,ios,mobile,chrome,united states,organic search,innovation,games,Video Gaming,Video Game Genres
8216,None,17909298741426735291,1611604237,2021-01-25,/sites/paultassi/2021/01/25/what-destiny-2-nee...,article-amp/standard/subscriber/standard,66,1,0.50,91.00,...,2021-01-25 10:07:28,ios,mobile,chrome,united states,organic search,innovation,games,Video Gaming,Video Game Genres
8459,None,17909298741426735291,1606536242,2020-11-27,/sites/paultassi/2020/08/09/the-top-24-destiny...,article-amp/standard/subscriber/standard,12,1,0.00,28.00,...,2020-08-09 10:09:20,ios,mobile,chrome,united states,organic search,innovation,games,Video Gaming,Video Game Genres
10721,None,17909298741426735291,1612367425,2021-02-03,/sites/paultassi/2021/02/03/here-is-the-destin...,article-amp/standard/subscriber/standard,76,1,0.50,112.00,...,2021-02-03 10:11:12,ios,mobile,chrome,united states,organic search,innovation,games,Video Gaming,Video Game Genres
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4288733,None,10277323451590763576,1624042093,2021-06-18,/sites/hughmcintyre/2021/06/18/bts-doja-cat-ju...,article-amp/standard/subscriber/standard,318,1,0.50,7.00,...,2021-06-18 09:30:00,ios,mobile,safari,united states,organic search,business,hollywood & entertainment,Music and Audio,None
4290825,None,16891615897367394640,1614631981,2021-03-01,/sites/robertlaszewski2/2021/03/01/the-democra...,article-amp/standard/subscriber/standard,2,1,0.50,96.00,...,2021-03-01 14:19:14,ios,mobile,safari (in-app),united states,content aggregators,business,policy,Family and Relationships,Eldercare
4291346,None,11754047996116205568,1624843039,2021-06-27,/sites/scottmendelson/2020/11/27/box-office-dr...,article-amp/standard/subscriber/standard,77,1,0.50,273.00,...,2020-11-27 13:10:00,ios,mobile,safari,united states,organic search,business,hollywood & entertainment,Pop Culture,None
4291846,None,11754047996116205568,1607366908,2020-12-07,/sites/scottmendelson/2013/08/18/weekend-box-o...,article-amp/standard/subscriber/standard,52,1,0.75,412.00,...,2013-08-18 11:41:00,ios,mobile,safari,united states,organic search,business,hollywood & entertainment,None,None


In [13]:
suspicious_fvid = nonsubs_data[nonsubs_data.GA_dfpNewZone.str.contains('/subscriber/')].GA_fullVisitorId.unique()
len(suspicious_fvid)

62

In [14]:
print("Piano ID for suspicious fvids?: ", nonsubs_data[nonsubs_data.GA_dfpNewZone.str.contains('/subscriber/')].piano_id.unique(), "\n")

# print("suspicious fvids present in subs dataset?: ", raw[raw.ga_fullvisitorid.isin(suspicious_fvid)].shape) # no

Piano ID for suspicious fvids?:  [None] 



In [15]:
# remove these suspicious fvids

print("Before: ", nonsubs_data.shape)

nonsubs_data = nonsubs_data[~nonsubs_data.GA_fullVisitorId.isin(suspicious_fvid)]

print("After: ", nonsubs_data.shape)

print("Unqiue non-subs fvids for eda: ", len(nonsubs_data.GA_fullVisitorId.unique()))

Before:  (4293776, 22)
After:  (4278128, 22)
Unqiue non-subs fvids for eda:  109938


* dfpNewZone meaning:

    * Default = not logged in, not subscribed
    * nonsubscriber = logged in but not subscribed

In [16]:
# now all clean

nonsubs_data['subs_from_GA_dfpNewZone'] = nonsubs_data['GA_dfpNewZone'].str.split('/').str[2]

nonsubs_data[['GA_fullVisitorId', 'subs_from_GA_dfpNewZone']].drop_duplicates(keep = 'first').subs_from_GA_dfpNewZone.value_counts()

default          109927
advisor            2405
nonsubscriber       507
advisoruk           206
Name: subs_from_GA_dfpNewZone, dtype: int64

In [ ]:
check = pd.DataFrame(nonsubs_data.groupby('GA_fullVisitorId').subs_from_GA_dfpNewZone.nunique()).reset_index()
print(check.shape)
multi = check[check.subs_from_GA_dfpNewZone>1]
print(nonsubs_data[nonsubs_data.GA_fullVisitorId =='9984661072098390819'].subs_from_GA_dfpNewZone.unique())
nonsubs_data[nonsubs_data.GA_fullVisitorId.isin(list(multi.GA_fullVisitorId))][['GA_fullVisitorId', 'subs_from_GA_dfpNewZone']].drop_duplicates(keep = 'first').subs_from_GA_dfpNewZone.value_counts()

* Curiosity: How many people in total with count(articles)>4 in a month since 2021-01-01?

In [24]:
start_time = time.time()

sql = """SELECT
                    ga_fullvisitorid, 
                    EXTRACT(MONTH FROM ga_date) AS ga_month,
                    MAX(ga_pianoId) AS piano_id,
                    count(GA_cmsNaturalId) AS num_articles
                FROM
                    `api-project-901373404215.DataMart.v_DataMart_updated`
                WHERE 
                    ga_date BETWEEN '2021-10-01' AND '2021-10-31' AND STARTS_WITH(GA_cmsNaturalId, "blogandpostid/blog/post/") AND ga_fullvisitorid NOT IN 
                        (
                        SELECT DISTINCT ga_fullvisitorid FROM `api-project-901373404215.skt.raw_subscriber`
                        )
                GROUP BY 
                    ga_fullvisitorid, ga_month
                HAVING 
                
                    # make sure the fvid never had a piano id and has read >4 articles in a month this year
                    
                    piano_id IS NULL AND 
                    num_articles > 4"""
mod = (
    bqclient.query(sql)
    .result()
    .to_dataframe(bqstorage_client=bqstorageclient)
) 

print("--- %s seconds ---" % (time.time() - start_time))

print(mod.shape)

--- 11.015136957168579 seconds ---
(402533, 4)


In [25]:
# almost 10mm eligible fvids

print(len(mod.ga_fullvisitorid.unique()))

10670002


In [36]:
# example - not everyone from these eligible pool came recently

mod[mod.ga_fullvisitorid =='14016339515174025045'].sort_values('ga_month')

,ga_fullvisitorid,ga_month,piano_id,num_articles
10255418,14016339515174025045,1,None,9
12081930,14016339515174025045,2,None,16
12422236,14016339515174025045,3,None,15
12287507,14016339515174025045,4,None,13
9050906,14016339515174025045,5,None,8


In [32]:
# say for prediction pipeline, eligible pool may be ~400k per month

mod[mod.ga_month==10]

,ga_fullvisitorid,ga_month,piano_id,num_articles
279,5245420159078807935,10,None,26
480,15042887427399443751,10,None,75
571,16671542372673828679,10,None,23
575,5478761675026730866,10,None,20
604,3949555784626988506,10,None,20
...,...,...,...,...
12789582,2071380040391217202,10,None,19
12789640,18273081965688604589,10,None,19
12789704,13328768785957695664,10,None,19
12789870,17996963963020610536,10,None,19


* What this may mean is collected pool of non-subscriber fvids have either 
    * used up free 5 articles but dint subscriber since 2021-01-01
    * Greg found out that piano is in charge of calculating who should see the paywall so it is possible that there are users slipping through the cracks as far as seeing too many articles since 2021-01-01

In [38]:
mod.num_articles.describe()

count   12789950.00
mean           8.28
std           13.41
min            5.00
25%            5.00
50%            6.00
75%            9.00
max        17745.00
Name: num_articles, dtype: float64

In [29]:
mod.ga_month.unique()

array([10])

In [ ]:
# all oct 100k pool present in mod
ns[ns.ga_fullvisitorid.isin(mod.ga_fullvisitorid)]

In [25]:
start_time = time.time()

query_string = """
  SELECT
       *
    FROM
        `api-project-901373404215.skt.raw_nonsubscriber`
"""
orig_ns = (
    bqclient.query(query_string)
    .result()
    .to_dataframe(bqstorage_client=bqstorageclient)
) 

print("--- %s seconds ---" % (time.time() - start_time))

print(orig_ns.shape)

--- 2.9483180046081543 seconds ---
(110000, 3)


In [28]:
orig_ns[orig_ns.ga_fullvisitorid.isin(ns.ga_fullvisitorid)]

,random_num,piano_id,ga_fullvisitorid
133,0.00,None,14948493918666874742
279,0.00,None,16839092622935256576
394,0.00,None,4146302520562400574
400,0.00,None,5112576306681197091
489,0.00,None,5153369622713654256
...,...,...,...
109494,0.01,None,512008667463409998
109510,0.01,None,2597822213911344685
109537,0.01,None,1710093749467569498
109574,0.01,None,118727386413760163
